In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from SyntheticControlMethods import Synth
from PIL import Image

In [5]:
Image.MAX_IMAGE_PIXELS = None

<h3> Guyana </h3>

As a preliminary example, we will attempt to plot the time-series of forest cover for Guyana based off of the Hansen Global Forest Change dataset. 

In [6]:
im = Image.open("/maps-priv/maps/og309/Hansen_GFC-2021-v1.9_treecover2000_10N_060W.tif")

In [9]:
imarray = np.array(im)

In [17]:
imarray.shape

(40000, 40000)